<a href="https://colab.research.google.com/github/jhmuller/nextWord/blob/main/nextWord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import re
import os
import string
import numpy as np
import sklearn

https://towardsdatascience.com/next-word-prediction-with-nlp-and-deep-learning-48b9fe0a17bf

In [14]:
from google.colab import drive
try:
  os.listdir("/content/drive")
except:
  drive.mount('/content/drive')
ddir = "/content/drive/MyDrive/data"
fname = 'lambada_development_plain_text.txt'
fpath = os.path.join(ddir, fname)
with open(fpath, 'r') as fp:
  orig_text = fp.read()

In [21]:
orig_text[:200]
clean_text = re.sub("\'|\,|\.|\?|\!", '', orig_text)
clean_text[:300]

'her pay for the evening was almost double that of the wait staff and although that might not seem like a lot to some people  it was a small fortune to claire  after loading her final tray for a server  claire went to the restroom to freshen up and begin preparations for being loaded into the cake  p'

In [2]:
URL = 'https://www.gutenberg.org/files/67138/67138-0.txt'
response = requests.get(URL)
orig_text = response.content

In [50]:
M = 2535
temp_text = str(orig_text[M:])
clean_text = re.sub("\\\\r|\\\\n|\\\\xbb|\\\\xef|\\\\xbf|x80|x9|xe|\d.", " ", temp_text)
cleaner_text = clean_text.translate(str.maketrans('', '', string.punctuation))

In [51]:
len(cleaner_text)
cleaner_text[:200]

'b    Robert Cohn was once middleweight boxing champion of Princeton Do not  think that I am very much impressed by that as a boxing title but it  meant a lot to Cohn He cared nothing for boxing in fac'

In [56]:
 
N = len(cleaner_text)
cutoff = int(np.floor(0.2*N))
while cleaner_text[cutoff] != ' ':
  cutoff += 1
train = cleaner_text[:cutoff]
test = cleaner_text[cutoff:]
test[:100]

' face down on the bed I was having a  bad time I heard them talking but I did not listen Brett came '

In [68]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts([train])

NGRAM_LEN = 4
def make_Xy(data, tokenizer, ngram_len=1):

  sequence_data = tokenizer.texts_to_sequences([data])[0]
  print(len(sequence_data))
  sequences = []
  X = []
  y = []
  for i in range(1, len(sequence_data)-ngram_len-1):
      words = sequence_data[i-1:i+ngram_len]
      X.append(words[:-1])
      y.append(words[-1])

  X = np.array(X)
  y = np.array(y)

  from  tensorflow.keras.utils import to_categorical
  vocab_size = len(tokenizer.word_index) + 1

  ycat = to_categorical(y, num_classes=vocab_size)
  return X, ycat, y

Xtrain, ytrain, yindstrain = make_Xy(train, tokenizer, ngram_len=NGRAM_LEN)
Xtest, ytest, yindstest = make_Xy(test, tokenizer, ngram_len=NGRAM_LEN)



14932
50776


In [69]:
print(tokenizer.sequences_to_texts([yindstest[:5]]))
tokenizer.sequences_to_texts(Xtest[:5])

['bed i was having a']


['face down on the',
 'down on the bed',
 'on the bed i',
 'the bed i was',
 'bed i was having']

In [70]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
def get_vocab_size(data):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts([data])
  vocab_size = len(tokenizer.word_index) + 1
  return vocab_size

train_vocab_size = get_vocab_size(train)
model = Sequential()
model.add(Embedding(train_vocab_size, 20, input_length=NGRAM_LEN))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(train_vocab_size, activation="softmax"))

In [71]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword1'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

In [73]:
from tensorflow.keras.optimizers import Adam
model.compile(loss="categorical_crossentropy", metrics=['accuracy'], 
              optimizer=Adam(learning_rate=0.001))
model.fit(Xtrain, ytrain, epochs=16, batch_size=64, callbacks=[reduce_lr])

Epoch 1/16
234/234 [==============================] - 6s 15ms/step - loss: 1.2417 - accuracy: 0.6540 - lr: 0.0010
Epoch 2/16
234/234 [==============================] - 3s 14ms/step - loss: 0.8046 - accuracy: 0.7788 - lr: 0.0010
Epoch 3/16
234/234 [==============================] - 3s 14ms/step - loss: 0.5980 - accuracy: 0.8332 - lr: 0.0010
Epoch 4/16
234/234 [==============================] - 3s 15ms/step - loss: 0.4387 - accuracy: 0.8869 - lr: 0.0010
Epoch 5/16
234/234 [==============================] - 3s 14ms/step - loss: 0.3508 - accuracy: 0.9118 - lr: 0.0010
Epoch 6/16
234/234 [==============================] - 3s 14ms/step - loss: 0.2618 - accuracy: 0.9372 - lr: 0.0010
Epoch 7/16
234/234 [==============================] - 3s 15ms/step - loss: 0.2199 - accuracy: 0.9504 - lr: 0.0010
Epoch 8/16
234/234 [==============================] - 3s 15ms/step - loss: 0.1908 - accuracy: 0.9567 - lr: 0.0010
Epoch 9/16
234/234 [==============================] - 3s 15ms/step - loss: 0.1788 - accu

In [79]:
x_data = Xtest
y_data = ytest

In [82]:
vpreds = model.predict(x_data[:20])
print(len(vpreds))
pred_inds = [np.argmax(x) for x in vpreds]
pred_words = tokenizer.sequences_to_texts([pred_inds])
pred_words

1/1 [==============================] - 0s 18ms/step
20


['stairs the felt tall a great of about e the be d he went not realize anything to on my']

In [83]:
true_inds = [np.argmax(x) for x in y_data[:20]]
true_words = tokenizer.sequences_to_texts([true_inds])
true_words

['bed i was having a bad time i heard them talking but i did not listen brett came in and']

In [78]:
triples_words = tokenizer.sequences_to_texts(x_data[:10])
triples_words

['b robert cohn was',
 'robert cohn was once',
 'cohn was once middleweight',
 'was once middleweight boxing',
 'once middleweight boxing champion',
 'middleweight boxing champion of',
 'boxing champion of princeton',
 'champion of princeton do',
 'of princeton do not',
 'princeton do not think']

In [21]:
len(ytest[0])

3172

In [95]:
pred_inds

[390, 247, 1, 122, 11, 14, 151, 5, 77, 376]